In [ ]:
!pip install fastapi nest-asyncio pyngrok uvicorn
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 13.0 MB/s eta 0:00:00


In [ ]:
# Importar las bibliotecas necesarias
from fastapi import FastAPI, File, UploadFile
from pydantic import BaseModel
from fastapi.middleware.cors import CORSMiddleware
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import cv2
import numpy as np
import json
import base64
import io
import uvicorn
from pyngrok import ngrok
import asyncio
import nest_asyncio
from concurrent.futures import ThreadPoolExecutor

In [ ]:
from google.colab import drive
# Montar Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Aplicar nest_asyncio para permitir el anidamiento de bucles de eventos
nest_asyncio.apply()

# Configurar el authtoken de ngrok
ngrok.set_auth_token("2jqS67zz1ACVJgG3plcESb0ssJO_5P9GqkrZiA4hr9BaeX3oX")


app = FastAPI()

# Configurar CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)


# Configuración
IMG_SIZE = 64

# Cargar el modelo y las etiquetas
model = load_model('/content/drive/MyDrive/Personales/Estudio/BigData/Mod11/TareaFinal/face_recognition_model.h5')

def load_labels(filename):
    with open(filename, 'r') as f:
        return json.load(f)

label_mapping = load_labels('/content/drive/MyDrive/Personales/Estudio/BigData/Mod11/TareaFinal/face_recognition_model.json')
label_names = {v: k for k, v in label_mapping.items()}


def preprocess_image(image, target_size=(64, 64)):
    # Convertir a RGB si es necesario
    if len(image.shape) == 2 or image.shape[2] == 1:
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    elif image.shape[2] == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Detección facial
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    if len(faces) > 0:
        x, y, w, h = faces[0]
        face = image[y:y+h, x:x+w]
    else:
        return None

    # Redimensionar
    face = cv2.resize(face, target_size)

    # Normalizar
    face = face.astype(np.float32) / 255.0

    return face

def identify_person(image):
    processed_img = preprocess_image(image, target_size=(IMG_SIZE, IMG_SIZE))

    if processed_img is None:
        return None, 0

    # Preparar la imagen para la predicción
    img_array = np.expand_dims(processed_img, axis=0)

    # Realizar la predicción
    prediction = model.predict(img_array)
    person_index = np.argmax(prediction)
    confidence = prediction[0][person_index]

    # Si la confianza es menor a 0.85, retornar None
    if confidence < 0.85:
        return None, confidence

    person_name = label_names[person_index]
    return person_name, confidence

class IdentificationResponse(BaseModel):
    person: str | None
    confidence: float
@app.post("/identify", response_model=IdentificationResponse)
async def identify_image(file: UploadFile = File(...)):
    contents = await file.read()
    nparr = np.fromstring(contents, np.uint8)
    img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

    person, confidence = identify_person(img)

    return IdentificationResponse(person=person, confidence=float(confidence))

# Configurar ngrok
ngrok_tunnel = ngrok.connect(8004)
print('URL pública:', ngrok_tunnel.public_url)

# Función para ejecutar el servidor
async def run_server():
    config = uvicorn.Config(app, host="0.0.0.0", port=8004, log_level="info")
    server = uvicorn.Server(config)
    await server.serve()

# Ejecutar el servidor en el bucle de eventos actual
loop = asyncio.get_event_loop()
with ThreadPoolExecutor() as executor:
    future = executor.submit(loop.run_until_complete, run_server())

    try:
        future.result()
    except KeyboardInterrupt:
        print("Servidor detenido")

URL pública: https://cb57-34-106-47-40.ngrok-free.app


INFO:     Started server process [181]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8004 (Press CTRL+C to quit)
ERROR:asyncio:Exception in callback BaseSelectorEventLoop._accept_connection(<function Ser...x7a381816bf40>, <socket.socke...0.0.0', 8004)>, None, <Server socke....0', 8004)>,)>, 2048, None)
handle: <Handle BaseSelectorEventLoop._accept_connection(<function Ser...x7a381816bf40>, <socket.socke...0.0.0', 8004)>, None, <Server socke....0', 8004)>,)>, 2048, None)>
Traceback (most recent call last):
  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
RuntimeError: cannot enter context: <_contextvars.Context object at 0x7a38988a83c0> is already entered
ERROR:asyncio:Exception in callback _SelectorSocketTransport._read_ready()
handle: <Handle _SelectorSocketTransport._read_ready()>
Traceback (most recent call last):
  File "/usr/lib/python

INFO:     181.115.186.26:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     181.115.186.26:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     181.115.186.26:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     181.115.186.26:0 - "GET /openapi.json HTTP/1.1" 200 OK


ERROR:asyncio:Exception in callback BaseAsyncIOLoop._handle_events(9, 1)
handle: <Handle BaseAsyncIOLoop._handle_events(9, 1)>
Traceback (most recent call last):
  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
RuntimeError: cannot enter context: <_contextvars.Context object at 0x7a38ba8a7580> is already entered
<ipython-input-9-f7059d6dea18>:87: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  nparr = np.fromstring(contents, np.uint8)


1/1 [==============================] - 0s 247ms/step
INFO:     181.115.186.26:0 - "POST /identify HTTP/1.1" 200 OK


ERROR:asyncio:Exception in callback BaseAsyncIOLoop._handle_events(9, 1)
handle: <Handle BaseAsyncIOLoop._handle_events(9, 1)>
Traceback (most recent call last):
  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
RuntimeError: cannot enter context: <_contextvars.Context object at 0x7a38ba8a7580> is already entered
